# A Multiagent transportation system

Auteur : Philippe Mathieu, [CRISTAL Lab](https://www.cristal.univ-lille.fr/), [SMAC Team](https://www.cristal.univ-lille.fr/?rubrique26&id=7), [University of Lille](http://www.univ-lille1.fr), email : philippe.mathieu@univ-lille.fr

Contributeurs : Corwyn Fèvre (CRISTAL/SMAC , CRISTAL/OSM)

Creation : 15/01/2020


## Principe général


Cette feuille fait suite à [mas_basics_fr.ipynb](mas_basics_fr.ipynb) qui donnait les bases de la construction d'un système multi-agents sur un reseau social.
Nous vous montrons ici, comment réaliser une petite simulation d'un système de co-voiturage à l'aide des agents.

## L'agent

Dans sa forme la plus simple, un agent est une entité dotée d'une seule
capacité, celle de décider quoi faire. Personne ne lui indique ce qu'il doit faire,
c'est lui qui décide ! C'est le principe d'**autonomie**. Il suffit juste de lui donner la
parole pour qu'il agisse. Lors de sa prise de parole, l'agent réalise sémantiquement 3 étapes différentes : la **perception** de son entourage, la **decision** en fonction de son propre état et de ce qu'il a perçu, puis l'**action** effective qu'il réalise in fine. Idéalement chaque agent ne peut faire qu'une seule action lors de sa prise de parole.

In [ ]:
class Agent:
      def __init__(self,name) :
          self.name=name
        
      def decide(self):
          print("Bonjour ! My name is "+self.name)

Il est bien sûr possible de créer plusieurs agents et de les interroger directement.

In [ ]:
a1 = Agent("philippe")
a2 = Agent("corwyn")
a1.decide()
a2.decide()

## Le système multiagent

Bien évidemment, un système multi-agent utilise des dizaines voire des
milliers d'agents. Il est alors necessaire de créer une classe
permettant de les manager. En général la méthode qui lance la
simulation se nomme `run` et prend en paramètre le nombre de prises de
paroles.
Afin d'assurer le principe d'**équité** on s'assurerque chaque agent a
au moins une fois la parole avant qu'un autre agent ne l'ait deux
fois. On s'appuie pour cela sur la notion de tour de parole. Chaque tour de
parole donne aléatoirement la parole à l'ensemble des agents, avant de
recommencer. Un tour de parole constitue sémantiquement une unité de temps, un tick d'horloge. La classe SMA s'écrit :


In [ ]:
import random
class SMA:
      def __init__(self):
        self.tick=0
        self.resetTicks()
        self.agentList = []

      def resetTicks(self):
          tick=0
          
      def addAgent(self,ag):
          self.agentList.append(ag)
 
      def run(self,rounds):
          for i in range(0,rounds):
              self.runOnce()

      def runOnce(self):
          self.tick+=1
          random.shuffle(self.agentList)
          for ag in self.agentList :
              ag.decide()
          print("tick "+str(self.tick)+" ended")

In [ ]:
sma = SMA()
sma.addAgent(Agent("paul"))
sma.addAgent(Agent("kim"))
sma.addAgent(Agent("Lisa"))
sma.run(5)

# Le problème
On a une grille. Un générateur de véhicules qui génère selon une loi de probabilité des véhicules sur l'un des axes dans l'un des sens. Simultanément apparaissent des passagers à un lei aléatoire pour une destination aléatoire (X,Y).
Les véhicules sont réputés toujours aller jusqu'au bout d'un axe et peuvent descendre un passager où ils veulent (c'est un bus ou un train). Le système est rythmé par unités de temps (ticks). Avancer d'une case prend 1 tick, et donc attendre un véhicule qui est 2 cases plus loin prend 2 ticks.


## L'environnement

In [ ]:
# Mettre ici la manière de coder l'environnement rudimentaire

## Les véhicules

In [ ]:
# mettre ici le code de la classe Vehicule, la plus simple possible (rasoir d'occam) 
# avec une petite description dans la case mardown juste au dessus


## Les générateurs de véhicules

In [ ]:
# Mettre ici le code d'un générateur trivial

# Une première version
Un passager est aveugle. Il monte dans le premier véhicule qui se présente dans sa direction (soit verticale soit horizontale) et il va jusqu'au maximum de sa possibilité (son X ou son Y de destination). Il se déplace dans une direction 1 fois, descend du véhicule, attend le prochain véhicule dans l'autre sens et arrive à sa destination.
Précisions :
- au maximum il attend 2 fois : au départ, et au changement
- grâce au rythme du système on est capable de tracer des courbes de temps d'attente (en tick)



# Une version plus poussée

Le passager perçoit une partie de l'environnement : sa verticale et son horizontale. Quand un véhicule arrive dans sa case, il regarde le nombre de cases N que ce véhicule lui permet de faire (N=X ou N=Y), et si il y a dans sa perception un autre véhicule à T distance qui lui permettra d'avancer d'un nombre de cases U (dans l'autre direction), avec U-T > N alors il attend celui là (Corwin raisonne sur un "potentiel" : U-T c'est le potentiel de cases dans l'autre direction avec un taux de transfert 0).
Une fois dans le véhicule, le passager fait ce raisonnement là à chaque case, et donc, décide de descendre ou pas, pour en attendre un autre.
Précisions :
- la formule de choix mise en place pourrait être inversée ! c'est une heuristique ! Il y a des heuristiques optimistes et des heuristiques pessimistes (ne serait-ce qu'en inversant l'inégalité).
- Dans cette version le passager peut donc voyager en zig-zag, néanmoins il ne "dépasse" jamais son X et son Y.


# Bibliographie

Philippe Mathieu, Yann Secq.
Environment Updating and Agent Scheduling Policies in Agent-based Simulators. 
ICAART Conference, 2012, pp 170-175

Philippe Mathieu, Sébastien Picault, Yann Secq.
Design Patterns for Environments in Multi-agent Simulations. 
PRIMA Conference, 2015, pp 678-686

Philippe Mathieu, Gildas Morvan, Sébastien Picault.
Multi-level agent-based simulations: Four design patterns. 
Journal of Simulation Modelling Practice and Theory, Janv 2018
